In [8]:
import pandas as pd
df = pd.read_csv('survey_results_public.csv', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 129 entries, Respondent to SurveyEasy
dtypes: float64(41), int64(1), object(87)
memory usage: 97.3+ MB


In [11]:
import numpy as np
category_columns = [
    column for column in df.columns if df[column].nunique() < 100 and df[column].dtype == np.dtype('O')
]
print(len(category_columns))

69


In [12]:
for column in category_columns:
    df[column] = df[column].astype('category')

In [17]:
import json
data_dtypes = {column:str(df[column].dtype) for column in df.columns}
json.dump(data_dtypes, open('data_dtypes.json', 'w'))

In [18]:
df = pd.read_csv('survey_results_public.csv', dtype=json.load(open('data_dtypes.json')), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 129 entries, Respondent to SurveyEasy
dtypes: category(69), float64(41), int64(1), object(18)
memory usage: 51.8+ MB


In [25]:
language_worked_with = df.LanguageWorkedWith.str.split(';', expand=True)
languages = set()
for values in language_worked_with.values:
    for value in values:
        if isinstance(value, str):
            languages.add(value)
languages = sorted(list(languages))
languages

['Assembly',
 'Bash/Shell',
 'C',
 'C#',
 'C++',
 'CSS',
 'Clojure',
 'Cobol',
 'CoffeeScript',
 'Delphi/Object Pascal',
 'Erlang',
 'F#',
 'Go',
 'Groovy',
 'HTML',
 'Hack',
 'Haskell',
 'Java',
 'JavaScript',
 'Julia',
 'Kotlin',
 'Lua',
 'Matlab',
 'Objective-C',
 'Ocaml',
 'PHP',
 'Perl',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VB.NET',
 'VBA',
 'Visual Basic 6']

In [27]:
def language_dummy_variable(languages):
    def _(x):
        x = set(x.tolist())
        return pd.Series([
            True if language in x else False
            for language in languages
        ])
    return _

sorted_language_worked_with = language_worked_with.apply(language_dummy_variable(languages), axis=1)
sorted_language_worked_with.columns = ['languageWorkedWith_' + language for language in languages]

In [40]:
sorted_language_worked_with.sum().sort_values(ascending=False)
# np.where(sorted_language_worked_with.sum().sort_values(ascending=False).index == 'languageWorkedWith_Python')

languageWorkedWith_JavaScript              54686
languageWorkedWith_HTML                    53628
languageWorkedWith_CSS                     50979
languageWorkedWith_SQL                     44670
languageWorkedWith_Java                    35521
languageWorkedWith_Bash/Shell              31172
languageWorkedWith_Python                  30359
languageWorkedWith_C#                      26954
languageWorkedWith_PHP                     24071
languageWorkedWith_C++                     19872
languageWorkedWith_C                       18042
languageWorkedWith_TypeScript              13626
languageWorkedWith_Ruby                     7911
languageWorkedWith_Swift                    6310
languageWorkedWith_Assembly                 5760
languageWorkedWith_Go                       5532
languageWorkedWith_Objective-C              5510
languageWorkedWith_VB.NET                   5254
languageWorkedWith_R                        4813
languageWorkedWith_Matlab                   4564
languageWorkedWith_V

In [41]:
df = pd.concat([df, sorted_language_worked_with], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 167 entries, Respondent to languageWorkedWith_Visual Basic 6
dtypes: bool(38), category(69), float64(41), int64(1), object(18)
memory usage: 55.4+ MB


In [43]:
def student_yes_or_no(x):
    if isinstance(x, float):
        return 'Unknown'
    elif x == 'No':
        return 'No'
    else:
        return 'Yes'

In [44]:
df['Student_1'] = df.Student.map(student_yes_or_no)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 168 entries, Respondent to Student_1
dtypes: bool(38), category(69), float64(41), int64(1), object(19)
memory usage: 56.1+ MB


In [55]:
hobby_years_coding_count = df[['Hobby', 'YearsCoding']].\
groupby(['Hobby', 'YearsCoding']).\
size().\
reset_index(name='counts').\
pivot(index='YearsCoding', columns='Hobby', values='counts')

hobby_years_coding_count

Hobby,No,Yes
YearsCoding,,
0-2 years,2293,8389
12-14 years,1524,6506
15-17 years,1217,4900
18-20 years,1019,4053
21-23 years,557,2091
24-26 years,365,1497
27-29 years,185,875
3-5 years,4295,19018
30 or more years,711,2833


In [56]:
hobby_years_coding_count.index = hobby_years_coding_count.index.set_categories(['0-2 years', '3-5 years', '6-8 years', '9-11 years', '12-14 years',
          '15-17 years', '18-20 years', '21-23 years',
          '24-26 years', '27-29 years', '30 or more years'
          ])
hobby_years_coding_count.sort_index(inplace=True)

In [57]:
hobby_years_coding_count

Hobby,No,Yes
YearsCoding,,
0-2 years,2293,8389
3-5 years,4295,19018
6-8 years,3395,15943
9-11 years,2208,9961
12-14 years,1524,6506
15-17 years,1217,4900
18-20 years,1019,4053
21-23 years,557,2091
24-26 years,365,1497
